# Models Trained on Tumor Images Only

In [1]:
# import necessary libraries
import numpy as np
from torch import nn
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torchvision.transforms as transforms
from IPython.display import display
from utils import *


Define Global Variable

In [ ]:
paper_transform = torchvision.transforms.Compose([
    torchvision.transforms.Grayscale(num_output_channels=1),
    torchvision.transforms.ToTensor() ])

In [ ]:
adj_transform = transforms.Compose([
    transforms.ToTensor(),  
    torchvision.transforms.Grayscale(num_output_channels=1),
    # Convert images to tensors
    transforms.Normalize((0.,), (1.,))  # Normalize image tensors
])

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Train and Evaluate Adjusted Model

In [ ]:
# create train, validition, and test sets
balanced_adj_train_set = torchvision.datasets.ImageFolder(
    'balanced_data/tumor_only/train',
    transform=adj_transform
)

balanced_adj_train_loader = torch.utils.data.DataLoader(
    balanced_adj_train_set,
    batch_size=128,
    shuffle=True,
    num_workers=0,  
    pin_memory=True,
)

balanced_adj_val_set = torchvision.datasets.ImageFolder(
    'balanced_data/tumor_only/validation',
    transform=adj_transform
)

balanced_adj_val_loader = torch.utils.data.DataLoader(
    balanced_adj_val_set,
    batch_size=128,
    shuffle=True,
    num_workers=0, 
    pin_memory=True,
)

balanced_adj_test_set = torchvision.datasets.ImageFolder(
    'balanced_data/tumor_only/test',
    transform=adj_transform
)

balanced_adj_test_loader = torch.utils.data.DataLoader(
    balanced_adj_test_set,
    batch_size=128,
    shuffle=True,
    num_workers=0,  
    pin_memory=True,
)

In [ ]:
# number of classes in the dataset
num_classes = len(balanced_adj_train_set.classes)

# create adjusted model
model = BrainTumorCNN(num_classes).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 20

training_losses = []
validation_losses = []
for epoch in range(epochs):
    train_loss, train_acc = train_model(model, balanced_adj_train_loader, loss_fn, optimizer)
    training_losses.append(train_loss)
    val_loss, val_acc = evaluate_model(model, balanced_adj_val_loader, loss_fn, device)
    validation_losses.append(val_loss)
    print(
        f"epoch: {epoch+1}/{epochs} | train loss={np.mean(train_loss):.4f}, {train_acc=:.4f} | val loss: {np.mean(val_loss):.4f}, {val_acc=:.4f}"
    )

In [ ]:
test_loss, test_acc, test_confusion_matrix = evaluate_model(
    model, balanced_adj_test_loader, loss_fn, device, return_confusion_matrix=True
)
print(f"{test_loss=}, {test_acc=}")
draw_confusion_matrix(test_confusion_matrix, balanced_adj_val_set, balanced_adj_train_set.classes)

Train and Evaluate Paper's Model

In [ ]:
# create train, validition, and test sets
balanced_pap_train_set = torchvision.datasets.ImageFolder(
    'balanced_data/tumor_only/train',
    transform=paper_transform
)

balanced_pap_train_loader = torch.utils.data.DataLoader(
    balanced_pap_train_set,
    batch_size=128,
    shuffle=True,
    num_workers=0,  
    pin_memory=True,
)

balanced_pap_val_set = torchvision.datasets.ImageFolder(
    'balanced_data/tumor_only/validation',
    transform=paper_transform
)

balanced_pap_val_loader = torch.utils.data.DataLoader(
    balanced_pap_val_set,
    batch_size=128,
    shuffle=True,
    num_workers=0, 
    pin_memory=True,
)

balanced_pap_test_set = torchvision.datasets.ImageFolder(
    'balanced_data/tumor_only/test',
    transform=paper_transform
)

balanced_pap_test_loader = torch.utils.data.DataLoader(
    balanced_pap_test_set,
    batch_size=128,
    shuffle=True,
    num_workers=0,  
    pin_memory=True,
)

In [ ]:
# paper's model
model = BrainTumorCLF().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 10

training_losses = []
validation_losses = []
for epoch in range(epochs):
    train_loss, train_acc = train_model(model, balanced_pap_train_loader, loss_fn, optimizer)
    training_losses.append(train_loss)
    val_loss, val_acc, val_precision, val_recall = evaluate_model(model, balanced_pap_val_loader, loss_fn, device)
    validation_losses.append(val_loss)
    print(
        f"epoch: {epoch+1}/{epochs} | train loss={np.mean(train_loss):.4f}, {train_acc=:.4f} | val loss: {np.mean(val_loss):.4f}, {val_acc=:.4f}"
    )

In [ ]:
import matplotlib.pyplot as plt

def plot_weight_histograms(model):
    # Count the number of convolutional and linear layers
    num_layers = sum(isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear) for _, layer in model.named_children())
    
    # Create a figure and axis for plotting
    fig, axs = plt.subplots(nrows=num_layers, ncols=1, figsize=(8, 6 * num_layers))

    # Track the current subplot index
    subplot_index = 0

    # Extract weights from each layer and plot histogram
    for layer_name, layer in model.named_children():
        # Check if the layer is a convolutional or linear layer
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            weights = layer.weight.detach().numpy().flatten()

            # Plot histogram of weights
            axs[subplot_index].hist(weights, bins=50)
            axs[subplot_index].set_title(f'{layer_name} Weight Histogram')
   
plot_weight_histograms(model)

In [ ]:
test_loss, test_acc, test_precision, test_recall, test_confusion_matrix = evaluate_model(
    model, balanced_pap_test_loader, loss_fn, device, return_confusion_matrix=True
)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_acc}, Test Precision: {test_precision}, Test Recall: {test_recall}")

confusion_matrix = draw_confusion_matrix(test_confusion_matrix, balanced_pap_val_set, balanced_pap_train_set.classes)